# Load in extracted features

In [37]:
# open the every file in a directory and print the first line of each file
import os
import sys
import numpy as np
import pandas as pd
from sklearn import preprocessing
from sklearn.metrics import accuracy_score
from sklearn.metrics import r2_score
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import BayesianRidge
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, SimpleRNN
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import randint


In [38]:
# load in the features
features_train = np.array([])
for file in os.listdir('features_train'):
    with open('features_train/' + file) as f:
        df = pd.read_csv(f)
        features_train = np.column_stack((features_train, df.values)) if features_train.size else df.values

# load the labels
train = pd.read_csv('train.csv')
y_train = train['Genre']
label_encoder = LabelEncoder()
y_train = label_encoder.fit_transform(y_train)
# normalize the data
features_train = preprocessing.scale(features_train)

# Load in test Data


In [39]:
features_test = np.array([])
for file in os.listdir('features_test'):
    with open('features_test/' + file) as f:
        df = pd.read_csv(f)
        features_test = np.column_stack((features_test, df.values)) if features_test.size else df.values

# scale the test data
features_test = preprocessing.scale(features_test)


# Logistic Regression

In [40]:

logreg = LogisticRegression(max_iter=150)
logreg.fit(features_train, y_train)
predictions = logreg.predict(features_train)
# Calculate the accuracy
accuracy = accuracy_score(y_train, predictions)
print("Logistic Regression accuracy:", accuracy)


Logistic Regression accuracy: 0.885


# Predict with Logistic Regression and save to CSV

In [41]:

# run the model on the test data
predictions = logreg.predict(features_test)
predictions = label_encoder.inverse_transform(predictions)
# save the predictions to a csv file
df = pd.DataFrame({'ID': os.listdir('test'), 'genre': predictions})
df.to_csv('predictionsLR.csv', index=False)

# Neural Network

In [45]:
model = Sequential()
model.add(Dense(80, activation='relu', input_shape = (65,), kernel_regularizer='l2', bias_regularizer='l2', activity_regularizer='l2'))

model.add(Dense(256, activation='relu'))

model.add(Dense(128, activation='relu'))

model.add(Dense(60, activation='softmax'))

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

model.fit(features_train, y_train, epochs=25, batch_size=28, validation_split=0.2, verbose=2, shuffle=True, steps_per_epoch=10, validation_steps=10, validation_batch_size=28, validation_freq=1, max_queue_size=10, workers=1, use_multiprocessing=False)

loss, accuracy = model.evaluate(features_train, y_train)
print('Neural Network accuracy:', accuracy)

TypeError: TensorFlowTrainer.fit() got an unexpected keyword argument 'max_queue_size'

# Predict with Neural Network and save to CSV

In [34]:
predictions = model.predict(features_test)
predictions = np.argmax(predictions, axis=1)
predictions = label_encoder.inverse_transform(predictions)


df = pd.DataFrame({'ID': os.listdir('test'), 'genre': predictions})
df.to_csv('predictionsNN.csv', index=False)

7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step


ValueError: y contains previously unseen labels: [15 27 30 32 34 38 46 57 58]

Random Forest

In [23]:
# use RandomForestClassifier and RandomizedSearchCV

# initialize the parameters which we want to optimize
params = {'n_estimators': randint(50, 200), 'max_depth': randint(1, 20)}
# print(randint(50, 200))
# Initialize Random Forest Classifier
rf = RandomForestClassifier()
# Initialize randomized search CV, which finds the best values for the given inputs
param_search = RandomizedSearchCV(rf, params)
param_search.fit(features_train, y_train)
best = param_search.best_estimator_
best_params = param_search.best_params_
# print(best_params)
# print(param_search.cv_results_)
# Get the best parameter values
max_depth = best_params['max_depth']
n_estimators = best_params['n_estimators']
# Create a random forest using these inputs
rf_best = RandomForestClassifier(n_estimators, max_depth=max_depth)
rf_best.fit(features_train, y_train)
# Predict using this classifier and then check our accuracy
predictions = rf_best.predict(features_train)
# Calculate the accuracy
accuracy = accuracy_score(y_train, predictions)
print(accuracy)

0.99875


In [24]:
predictions = rf_best.predict(features_test)
predictions = label_encoder.inverse_transform(predictions)
# save the predictions to a csv file
df = pd.DataFrame({'ID': os.listdir('test'), 'genre': predictions})
df.to_csv('predictionsRF.csv', index=False)